In [105]:
import os

In [106]:
%pwd

'/Users/vishalchandru/MacBook/KidneyCNN'

In [108]:
os.chdir('/Users/vishalchandru/MacBook/KidneyCNN')

In [109]:
%pwd

'/Users/vishalchandru/MacBook/KidneyCNN'

In [110]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/VishalChandru/KidneyCNN.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="VishalChandru"
os.environ["MLFLOW_TRACKING_PASSWORD"]="db7fb84a8a680ae7d4b6ee54f6ad4bf9e542dc0f"

In [111]:
import tensorflow as tf

In [112]:
model = tf.keras.models.load_model("artifacts/model_training/model.h5")

In [113]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_param: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [114]:
from Kidney_CTScan.constants import *
from Kidney_CTScan.utils.utility import read_yaml, create_directories, save_json
from Kidney_CTScan import logger, CustomException
import sys

In [115]:
class ConfigurationManager():
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAM_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        model_evaluation_config = ModelEvaluationConfig(
                                    path_of_model='artifacts/model_training/model.h5',
                                    training_data='artifacts/data_ingestion/KidneyCTScan',
                                    mlflow_uri='https://dagshub.com/VishalChandru/KidneyCNN.mlflow',
                                    all_param=self.params,
                                    params_image_size=self.params.IMAGE_SIZE,
                                    params_batch_size=self.params.BATCH_SIZE
                                    )
        return model_evaluation_config

In [116]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [117]:
class Evaluation:
    def __init__(self, config = ModelEvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.30
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        #generate validation data
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = "validation",
            shuffle = False,
            **dataflow_kwargs
        )
    
    def evaluate(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(file_path = Path("scores.json"), data= scores)
    
    def log_into_mlflow(self):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_param)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            if tracking_url_type_store != "file":
                mlflow.log_artifact(self.config.path_of_model, "model")
            else:
                mlflow.log_artifact(self.config.path_of_model, "model")

    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

In [118]:
try:
    config = ConfigurationManager()
    eval_config = config.get_model_evaluation_config()
    evaluation = Evaluation(config = eval_config)
    evaluation.evaluate()
    evaluation.log_into_mlflow()
except Exception as e:
    raise CustomException(e, sys)

[ 2023-12-02 19:24:21,662 ] INFO : utility - 20 - Read the yaml file from config/config.yaml
[ 2023-12-02 19:24:21,663 ] INFO : utility - 20 - Read the yaml file from params.yaml
[ 2023-12-02 19:24:21,663 ] INFO : utility - 31 - Created a directory on artifacts
Found 318 images belonging to 2 classes.
20/20 [==============================] - 35s 2s/step - loss: 1.5356 - accuracy: 0.6352
[ 2023-12-02 19:24:56,681 ] INFO : utility - 40 - Create json file at scores.json
